In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
from collections import Counter
import numpy as np
from numpy import loadtxt
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
p_scores = pd.read_csv('merfish_M1S1_filtered_periph_scores.csv')
p_scores = p_scores.loc[p_scores['annotation'] != 'unannotated']

In [ ]:
# intensize computation: results stored in periphery_data.npy
num_unique_cells = (p_scores['cell_id']).unique().shape[0]
num_genes = (p_scores['gene']).unique().shape[0]
print("There are", num_genes, "genes")
print("There are", num_unique_cells, "unique cells")

annotation_indices = {k: v for v, k in enumerate(list(p_scores['annotation'].unique()))}
cell_indices = {k: v for v, k in enumerate(list(p_scores['cell_id'].unique()))}
gene_indices = {k: v for v, k in enumerate(list(p_scores['gene'].unique()))}

In [ ]:
# intensize computation: results stored in periphery_labels.npy
# x data shape(num_unique_cells, gene_periphery_scores)
data = np.zeros((num_unique_cells, num_genes))
for cell in cell_indices:
    subset = p_scores.loc[p_scores['cell_id'] == cell]
    for index, s in subset.iterrows():
        cell_index = cell_indices[cell]
        gene_index = gene_indices[s['gene']]
        data[cell_index][gene_index] = s['periphery_score']

In [ ]:
np.save('periphery_data', data)
np.save('periphery_labels', labels)

In [ ]:
# y data 
labels = np.zeros((num_unique_cells, len(annotation_indices)))
for cell in cell_indices:
    ann = list(p_scores.loc[p_scores['cell_id'] == cell]['annotation'])[0]
    arr_index = annotation_indices[ann]
    labels[cell_indices[cell]][arr_index] = 1

In [ ]:
data = np.load('periphery_data.npy')
labels = np.load('labels.npy')
print(data.shape)
print(labels.shape)

In [ ]:
# cv = np.std(data, axis=0) / np.mean(data, axis=0)
# data = data[:,np.logical_and(cv >= np.percentile(cv, 1), cv <= np.percentile(cv, 99))]

In [ ]:
expr = np.sum(np.square(data), axis=1)
d = data[np.logical_and(expr >= np.percentile(expr, 1), expr <= np.percentile(expr, 99)),]
# l = labels[np.logical_and(expr >= np.percentile(expr, 1), expr <= np.percentile(expr, 99)),]
cv = np.std(total_set, axis=1) / np.mean(total_set, axis=1)
# total_set = total_set[np.logical_and(cv >= np.percentile(cv, 1), cv <= np.percentile(cv, 99)),]
# total_labels = total_labels[np.logical_and(cv >= np.percentile(cv, 1), cv <= np.percentile(cv, 99)),]

In [ ]:
d.shape

In [ ]:
l.shape

In [ ]:
# split data into train, val and test datasets
train, test, train_y, test_y = train_test_split(d, l, test_size=0.3)
test, val, test_y, val_y = train_test_split(test, test_y, test_size=0.5)
print(train.shape, train_y.shape)
print(val.shape, val_y.shape)
print(test.shape, test_y.shape)

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Input(192),
                                    tf.keras.layers.Dense(100, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(50, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(25, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(80, activation=tf.nn.softmax)])
lr= tf.keras.optimizers.schedules.ExponentialDecay(
    0.001,
    decay_steps=100000,
    decay_rate=0.95,
    staircase=True)
model.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])
model.summary()
model.fit(train, train_y, epochs=300, validation_data=(val, val_y))

In [ ]:
model.evaluate(test, test_y)

In [ ]:
y_pred.shape

In [ ]:
np.where(y_pred

In [ ]:
print(test_y.shape)
print(y_pred.shape)
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(test_y, axis=1)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)